# Gaussian Quadrature

Here we consider the problem of efficiently estimating expected value integrals for a nonlinear function $f(\pmb{x}) : \mathbb{R}^n \to \mathbb{R}^m$
\begin{equation}
E[f(\pmb{x})] = \int_{\mathbb{R}^n} f(\pmb{x}) N(\pmb{x} | \pmb{0}, I) d \pmb{x}.
\end{equation}
Here the notation $N(\pmb{x} | \pmb{0}, I)$ is shorthand for the Gaussian probability density function with mean $\pmb{0}$ and identity covariance matrix, evaluated at the point $\pmb{x}$. In Bayesian filtering applications, the Gaussian weighting function is also referred to as the prior distribution. 

General Gaussian weight functions $N(\pmb{x} | \pmb{x_0}, P_x)$ are handled by changing variables
\begin{equation}
\int_{\mathbb{R}^n} f(\pmb{x}) N(\pmb{x} | \pmb{x_0}, P_x) d \pmb{x} = \int f(\pmb{x_0} + \sqrt{P_x} \chi) N(\pmb{\chi} | \pmb{0}, I) d \pmb{x}
\end{equation}
where $\sqrt{P_x}$ is a matrix square root (typically obtained by Cholesky factorization) of the covariance matrix $P_x$. We will look at quadrature rules of the form 
\begin{equation}
 E[f(\pmb{x})] \approx Q[f(\pmb{x})] = \sum w_i f(\pmb{\chi_i})
\end{equation}
with quadrature points $\pmb{\chi_i}$, also called sigma points, and associated weights $w_i$. 

## Gauss-Hermite Quadrature

In 1D, the Gauss-Hermite quadrature rule estimates the integral 

\begin{equation}

\end{equation}


## The Unscented Transform as a Quadrature Rule

A quadrature rule of degree $d$ can exactly integrate all monomial (and thus polynomial) functions up to and including degree $d$. A monomial of degree $d$ refers to a function $x_1^{N_1} x_2^{N_2} \cdots x_n^{N_n}$ where the $N_i$ are non-negative integers that sum to $d$. The unscented transform, described in [1],  is an efficient quadrature rule of degree $d=3$. There a number of formulations of the unscented transform, but we'll look at a commonly used set of $2n + 1$ sigma points, sometimes called Julier sigma points, given by

\begin{equation}
\begin{gathered}
\pmb{\chi_0} = \pmb{x_0} \\
\pmb{\chi_i} = \pmb{x_0} + \sqrt{n + \kappa} [\sqrt{P_x}]_i \\
\pmb{\chi_{i+n}} = \pmb{x_0} - \sqrt{n + \kappa} [\sqrt{P_x}]_i \\
i = 1, \cdots, n
\end{gathered}
\end{equation}
with associated weights

\begin{equation}
w_i =
\begin{cases} 
      \kappa / (n + \kappa) & i = 0 \\
      1/2(n + \kappa) & \text{otherwise}.
\end{cases}
\end{equation}
The notation $[\sqrt{P_x}]_i$ refers to the $i$-th row of the matrix square root of $P_x$.

Let's look at a particular example. Suppose that 
\begin{equation}
f([x_0, x_1, x_2]^T) = [x_1 x_2 + 1, \: x_2^2, \: 5 x_0 x_1 x_2 + 2 x_1]^T
\end{equation}
Below we compute the expected value $ E[f(\pmb{x})]$ using both random sampling and the unscented transform using the Julier sigma points.


In [1]:
import numpy as np
from numpy.random import multivariate_normal
from filterpy.kalman.sigmas import SigmaPoints

# Nonlinear function
def f(X):
    return np.array([X[1,:]*X[2,:] + 1., X[2,:]**2, 5.*X[0,:]*X[1,:]*X[2,:] + 2.*X[1,:]])

# Compute expected value via random sampling
samples = f(multivariate_normal(np.zeros(3), np.eye(3), 5900).T)
y_mean1 = samples.mean(axis = 1)

# Compute expected value using the UT
points = SigmaPoints()
# Julier sigma points
X, wm, wc = points.get_set(np.zeros(3), np.eye(3), 'julier')
y_mean2 = f(X) @ wm

print("Expected value from random sampling: {}".format(y_mean1))
print("Expected value from UT: {}".format(y_mean2))

Expected value from random sampling: [ 1.02151051  1.01807631 -0.08998407]
Expected value from UT: [1. 1. 0.]


In this case, the unscented transform computes the expected value since the $f(\pmb{x})$ has terms involving polynomials of at most degree 3. 